In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import math
%matplotlib inline
from scipy.stats.mstats import winsorize 

In [ ]:
## Load in a GeoJSON file containing the geometry information for US counties, where feature.id is a FIPS code. 
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

# 1. EIA fossil fuel power plant emissions data
The EIA has data on the annual emissions of each fossil fuel power plant in the U.S., available at https://www.eia.gov/electricity/data/emissions/. Documentation for this data is availaale here: https://www.eia.gov/electricity/data/guide/pdf/guide.pdf. This data is obtained through the annual EIA-860 and EIA-923 surveys and covers 7 'sectors' of plant, including both commercial and industrial facilities that are merely connected to the grid and can sell power. Because we are considering the emissions associated with jobs in the fossil fuel electricty generation industry, commercial and industrial facilities should be excluded, as the jobs at these facilities are likely to be captured as manufacturing jobs, commercial jobs, etc. in employment databases and not classified as power generation jobs, which would inflate emissions intensities for the reported fossil fuel power generation workers in a given region.

## 1.1 Import and clean emissions data

In [ ]:
# Import EIA fossil-fuel plant emissions data (https://www.eia.gov/electricity/data/emissions/)
emissions2019_raw = pd.read_excel(
    '../Input/emissions2019.xlsx',
    sheet_name=0,
    header=1,
    dtype={'Plant Code': str, 'Sector Code': str}
)

# Clean excel read
emissions2019_raw = emissions2019_raw[:-2]

# Drop columns and group by key descriptive fields
emissions2019_raw = emissions2019_raw[
    ['Plant Code', 'Plant Name', 'State', 'Generation (kWh)', 'Sector Code', 'Aggregated Fuel Group',
     'Useful Thermal Output (MMBtu)', 'Total Fuel Consumption (MMBtu)',
     'Fuel Consumption for Electric Generation (MMBtu)',
     '\n Fuel Consumption for Useful Thermal Output (MMBtu)',
     'Quantity of Fuel Consumed', 'Fuel Units', 'Tons of CO2 Emissions',
     'Metric Tonnes of CO2 Emissions', 'NERC Region',
     'Balancing Authority Code', 'Balancing Authority Name',
     'EIA Balancing Authority Region']
].groupby(
    by=['Plant Code', 'Plant Name', 'State', 'NERC Region', 'Sector Code',
        'Aggregated Fuel Group', 'Balancing Authority Name'],
    dropna=False,
    as_index=False).sum()

# Print number of entries
print('Number of entries (fossil fuel plants) in raw EIA emissions data:', len(emissions2019_raw))

# Note: difference between unique values of Plant Name after grouping and len of df is due to ~50 unspecified areas with the same
# Plant Name but different descriptive fields (e.g. different states)

The EIA emissions data lists plants multiple times if they use several different fuel types (once per fuel type). Therefore, to ensure that each plant is only listed once, should group these multiple listings in one listing, and set the fuel type classification according to what the primary fuel type is for that plant.

In [ ]:
# For each plant where two or more fuel types are used, keep fuel group classification that makes up the majority of generation for that plant
#  Create dataframe that maps Plant Code to the fuel classification that makes up the majority of that plant's generation ('primary fuel group')
primary_fuel = pd.DataFrame(
    emissions2019_raw.sort_values(
        'Generation (kWh)', ascending=False
    ).drop_duplicates([
        'Plant Code']).sort_index()
    [['Plant Code', 'Aggregated Fuel Group']]
)

#  Group emissions2019_raw by the same columns as above, minus aggregated fuel group
emissions2019_raw = emissions2019_raw.groupby(
    by=['Plant Code', 'Plant Name', 'State', 'NERC Region', 'Sector Code', 'Balancing Authority Name'],
    dropna=False,
    as_index=False
).sum()

#  Merge primary fuel group onto updated emissions dataframe
emissions2019_raw = pd.merge(emissions2019_raw, primary_fuel, how='left', on='Plant Code')

Now, can add nameplate capacity from generating unit-level data from EIA 860.

In [ ]:
# Import EIA generator data from Form 860 (https://www.eia.gov/electricity/data/eia860/)
# and use to extract Nameplate Capacity of all the plants considered by EIA
#  Read in data for each generating unit
gen = pd.read_excel('../Input/eia860/2019/3_1_Generator_Y2019.xlsx',
                    header=1,
                    dtype={'Utility ID': str, 'Plant Code': str})

#  Group data to plant-level, aggregating capacity
eia_plantcapacity = gen[['Plant Code', 'Nameplate Capacity (MW)']].groupby(
    by='Plant Code', as_index=False).sum()

# Merge nameplate capacity onto plant name in emissions2019 df
emissions2019 = pd.merge(emissions2019_raw, eia_plantcapacity, how='left', on='Plant Code')

## 1.2 Add additional qualitative fields

In [ ]:
# Import plant locations
plant_details2019 = pd.read_excel(
    open('../Input/eia860/2019/2___Plant_Y2019.xlsx', 'rb'),
    header=1,
    usecols='A:N',
    dtype={'Utility ID': str, 'Plant Code': str, 'Zip': str, 'County': str}
)

# Fix errors in county names, lower case of all strings
plant_details2019['County'] = plant_details2019['County'].str.lower()
plant_details2019['County'] = plant_details2019['County'].replace('autagua', 'autauga')

# Merge plant details onto emissions data
emissions2019 = pd.merge(
    emissions2019,
    plant_details2019[['Utility ID', 'Utility Name',
                       'Plant Code', 'Zip', 'County', 'Latitude', 'Longitude']],
    how='left', on='Plant Code'
)

# Read in county fips labels [NOTE: this file has been edited from that
#  used in other code to ensure county names and their formatting match]
fips = pd.read_csv('../Temp/fips_edited.csv', encoding="ISO-8859-1",
                   usecols=[1, 2, 3, 4], names=[
                       'FIPS', 'County', 'State Name', 'State'], dtype={'FIPS': str})
fips['County'] = fips['County'].str.lower()

# Merge FIPS codes onto emissions2019 df
emissions2019 = pd.merge(
    emissions2019,
    fips[['FIPS', 'County', 'State']],
    how='left',
    on=['County', 'State']
)

# Drop partially reported facilities (plant code 99999)
emissions2019 = emissions2019[emissions2019['Plant Code'] != '99999']

# Replace FIPS 02270 Wade hampton (AK) with 02158 Kusilvak, and 46113 Shannon with 46102 Oglala lakota, according to July 2015 changes
for i in np.arange(len(emissions2019)):
    if emissions2019.loc[i, 'FIPS'] == '02270':
        emissions2019.loc[i, 'FIPS'] = '02158'
        emissions2019.loc[i, 'County'] = 'kusilvak'
    elif emissions2019.loc[i, 'FIPS'] == '46113':
        emissions2019.loc[i, 'FIPS'] = '46102'
        emissions2019.loc[i, 'County'] = 'oglala lakota'

# Print total CO2e emissiosn from fossil fuel power plants
print('Total CO2e emissions from fossil fuel power plants in 2019:',
      np.round(emissions2019['Tons of CO2 Emissions'].sum() * 10**-6, 2),
      'MM tons CO2e')

## 1.3 Group data at the county level

In [ ]:
# Create column for emissions of each fuel type, so that county-level data has total emissions from each fuel type
for fuel in emissions2019['Aggregated Fuel Group'].unique():
    emissions2019[f'tonCO2e_{fuel}'] = emissions2019.apply(
        lambda x: x['Tons of CO2 Emissions'] if x['Aggregated Fuel Group'] == fuel else 0,
        axis=1
    )
emissions2019['tonCO2e_other'] = emissions2019['tonCO2e_MSW'] + emissions2019['tonCO2e_GEO']
emissions2019 = emissions2019.drop(columns=['tonCO2e_MSW', 'tonCO2e_GEO'])

cols = ['Plant Name', 'State', 'Sector Code', 'Generation (kWh)',
        'Tons of CO2 Emissions', 'tonCO2e_PET', 'tonCO2e_GAS',
        'tonCO2e_COAL', 'tonCO2e_other', 'tonCO2e_sum']
emissions2019['tonCO2e_sum'] = emissions2019['tonCO2e_PET'] + emissions2019['tonCO2e_GAS'] + \
    emissions2019['tonCO2e_COAL'] + emissions2019['tonCO2e_other']

# Add Establishments count for each plant, to be grouped when grouping at the county level
emissions2019['Establishments'] = np.ones(len(emissions2019))

# There are some datapoints that have different NERC Regions for the same FIPS. Manually overwrite these.
fips_nerc_crosswalk = pd.read_excel('../Temp/fips_nerc_crosswalk.xlsx', dtype={
                                    'FIPS': str}).drop_duplicates()

emissions2019 = pd.merge(emissions2019, fips_nerc_crosswalk, how='left', on='FIPS')
emissions2019['NERC Region'] = emissions2019.apply(
    lambda x: x['NERC Region_x'] if pd.isnull(x['NERC Region_y']) else x['NERC Region_y'], axis=1)
emissions2019 = emissions2019.drop(columns=['NERC Region_x', 'NERC Region_y'])

# Fix points with NERC Region == NaN
emissions2019['NERC Region'] = emissions2019.apply(
    lambda x: 'ASCC' if x.State == 'AK' and pd.isna(x['NERC Region']) else x['NERC Region'], axis=1)
emissions2019['NERC Region'] = emissions2019.apply(
    lambda x: 'HICC' if x.State == 'HI' and pd.isna(x['NERC Region']) else x['NERC Region'], axis=1)

emissions2019_county_fuels = emissions2019.groupby(
    by=['FIPS', 'County', 'State', 'Aggregated Fuel Group', 'NERC Region'],
    as_index=False
).sum()
emissions2019_county = emissions2019_county_fuels[
    ['FIPS', 'County', 'State', 'Generation (kWh)', 'Useful Thermal Output (MMBtu)', 'Tons of CO2 Emissions',
     'Metric Tonnes of CO2 Emissions', 'Nameplate Capacity (MW)', 'Establishments', 'tonCO2e_PET', 'NERC Region',
     'tonCO2e_GAS', 'tonCO2e_COAL', 'tonCO2e_other', 'tonCO2e_sum']
].groupby(by=['FIPS', 'County', 'State', 'NERC Region'], as_index=False).sum()

## 1.4 Fix Obvious Errors
From the above, Coconino county FIPS 04005 has one of the highest carbon footprints but a reported total nameplate capacity of 0. This is because the nameplate capacity field in the dataframe is listed as NaN.

In [ ]:
# Check NaNs in all fields
emissions2019.isna().sum()

In [ ]:
# Download plant codes of plants for which capacity is NaN into a csv file
emissions2019[emissions2019['Nameplate Capacity (MW)'].isna(
)]['Plant Code'].to_csv('../Temp/no_capacity_plants.csv')

### Manually add capacity data for these 37 plants in the csv file and re-read it in

All 'NaN' plants were found in the 'Retired and Canceled' sheet of the '3_1_Generator_Y2019.xlsx' excel file. Their capacities were manually inputted into the csv file. Now, re-read this csv file in and merge these capacities onto emissions2019 dataframe

In [ ]:
# Read in found plant capacities
no_capacity_plants_edited = pd.read_csv(
    '../Temp/no_capacity_plants_edited.csv',
    dtype={'Plant Code': str}
)

# Merge new plant capacities onto emissions2019 dataframe
emissions2019 = pd.merge(emissions2019, no_capacity_plants_edited, how='left', on='Plant Code')
emissions2019['Nameplate Capacity (MW)'] = emissions2019['Nameplate Capacity (MW)_x'].fillna(
    emissions2019['Nameplate Capacity (MW)_y'])
emissions2019 = emissions2019.drop(
    ['Nameplate Capacity (MW)_x', 'Nameplate Capacity (MW)_y'], axis=1)

# Re-compute county-level emissions
emissions2019_county_fuels = emissions2019.groupby(
    by=['FIPS', 'County', 'State', 'Aggregated Fuel Group', 'NERC Region'],
    as_index=False
).sum()
emissions2019_county = emissions2019_county_fuels[
    ['FIPS', 'County', 'State', 'Generation (kWh)', 'Useful Thermal Output (MMBtu)', 'Tons of CO2 Emissions',
     'Metric Tonnes of CO2 Emissions', 'Nameplate Capacity (MW)', 'Establishments', 'tonCO2e_PET',
     'tonCO2e_GAS', 'tonCO2e_COAL', 'tonCO2e_other', 'NERC Region']
].groupby(by=['FIPS', 'County', 'State', 'NERC Region'], as_index=False).sum()

# take log10 of absolute emissions for visualization purposes
emissions2019_county['Tons of CO2 Emissions (log10)'] = np.log10(
    emissions2019_county['Tons of CO2 Emissions'])

# Export power plant emissions as csv
emissions2019.to_csv('../Output/powerplant_emissions2019.csv')

# 2. Employment data and emissions per employee
The U.S. Energy & Employment Report (USEER) is a initiative from EFI, NASEO and BW Research, and reports U.S. energy jobs. At https://www.usenergyjobs.org/research users can download county-level data on U.S. energy jobs, including electric power generation, separated by fuel type. Data is obtained using a stratified sampling approach from the most recent iteration of the BLS QCEW along with a supplemental survey conducted by BW Research. This means that some of the data is imputed, which is where the non-integer results come from, as well as negative employment figures and very small employment figures that occur in some counties. More details on methodology here: https://static1.squarespace.com/static/5a98cf80ec4eb7c5cd928c61/t/5c7f3ac94e17b642f18c1edc/1551841994076/USEER_Appendix.pdf<br>
Additionally, the County Business Patterns is an annual data series (most recent: 2020) sourced from the Census Bureau and other federal agencies that provides subnational economic data by industry for >6 million single-unit establishment and 1.8 million multi-unit establishments. This data is available at the county level at up to 6-digit NAICS granularity. Data fields with less than 3 contributing establishments are witheld - therefore, NAICS granularity can vary.<br>

Because USEER data is imputed, some counties have large emissions figures but decimal employment figures between 0 and 1, which results in unrealistically high emissions intensities. To offset this somewhat, where county-level CBP2020 employment data for NAICS 221112 (fossil fuel electric power generation) is available it *could* be used to supplement (i.e. used in place of) county-level employment figures from USEER.

## 2.1 Read in USEER employment data

In [ ]:
# Read in USEER data
useer2019 = pd.read_excel(
    '../Input/USEER/USEER+2019+County+Employment1.xlsx',
    header=1,
    dtype={'County': str}
)
useer2019 = useer2019.rename(
    columns={'County': 'FIPS', 'Coal': 'emp_coal', 'Nat Gas': 'emp_ng',
             'Oil & Other Fossil Fuels': 'emp_oil&otherfossil'}
)

# Compute total fossil fuel EPG employment per county
useer2019['emp_totalfossilEPG'] = useer2019['emp_ng'] + \
    useer2019['emp_coal'] + useer2019['emp_oil&otherfossil']

## 2.2 Add employment data to county-level emissions data

In [ ]:
# Merge USEER employment data onto county-fuel emissions data
emissions2019_county_fuels_emp = pd.merge(
    emissions2019_county_fuels,
    useer2019[['FIPS', 'State', 'emp_ng', 'emp_coal', 'emp_oil&otherfossil']],
    how='left',
    on=['FIPS', 'State']
)

# Extract employment data for each fuel type
emissions2019_county_fuels_emp['emp'] = np.zeros(len(emissions2019_county_fuels_emp))

for i in np.arange(len(emissions2019_county_fuels_emp)):
    if emissions2019_county_fuels_emp.loc[i, 'Aggregated Fuel Group'] == 'GAS':
        emissions2019_county_fuels_emp.loc[i,'emp'] = emissions2019_county_fuels_emp.loc[i, 'emp_ng']
    elif emissions2019_county_fuels_emp.loc[i, 'Aggregated Fuel Group'] == 'COAL':
        emissions2019_county_fuels_emp.loc[i,'emp'] = emissions2019_county_fuels_emp.loc[i, 'emp_coal']
    else:
        emissions2019_county_fuels_emp.loc[i,'emp'] = emissions2019_county_fuels_emp.loc[i, 'emp_oil&otherfossil']

emissions2019_county_fuels_emp = emissions2019_county_fuels_emp.drop(
    columns=['emp_ng', 'emp_coal', 'emp_oil&otherfossil'])

# Fill NaN emp fields with 1 employee
emissions2019_county_fuels_emp = emissions2019_county_fuels_emp.fillna(1)

## 2.3 Clean employment data

### 2.3.1 Winsorize data
The USEER data, since it is partially imputed, contains several points where the employment figures do not match what we would reasonably expect given a plant' or county's generation, capacity or emissions data. For example, many counties' employment as reported by USEER are less than 1 employee, often in counties where there is significant power generation activity, while others might have employment figures greater than 1 but still far lower than we would expect given the power generation in the region. Because of this, data was winsorized to rationalize these errors. Generally, it is preferable to winsorize inputs variables rather than the outcome variable, which in this case is carbon intensity per employee. Three different types of winsorization were considered:
1. <b>Winsorizing county-level employment so that all counties with electric power generation report at least 1 employee.</b> This ensure that negative employment values and values between 0 and 1 are fixed.
2. <b>Calculating nameplate capacity and annual generation per employee (in MW/employee and MWh/employee, respectively), and winsorizing these distributions.</b> We would expect the number of employees at a plant to be roughly proportional to the size of the plant, which can be indicated by the capacity or generation of the plant. These ratios of plant size to employment likely different amongst fuel groups, so these indicators will be computed for each fuel group separately.
3. <b>Winsorizing county-level carbon intensity per employee.</b> Even after the above winsorizations, there are still one or two counties with extremely high carbon intensities. Winsorizing carbon intensity at the 99th percentile rectifies these anomalies without significantly altering the distribution of the outcome variable.

In all cases, these parameters were used to derive revised employment estimates for each county, leaving the emissions data (which were not derived using imputation) intact.

#### 2.3.1.1 Winsorize employment at 1 employee minimum per county

In [ ]:
# Winsorize employment data at 1 employee per entry (i.e. set any emp values less than 1 to 1)
print('No. datapoints winsorized to set employment to 1:',
      len(emissions2019_county_fuels_emp[emissions2019_county_fuels_emp['emp'] < 1]),
      'of', len(emissions2019_county_fuels_emp))
emissions2019_county_fuels_emp['emp_new'] = emissions2019_county_fuels_emp['emp'].apply(
    lambda x: 1 if x < 1 else x)
emissions2019_county_fuels_emp['Winsorized_emp?'] = emissions2019_county_fuels_emp['emp_new'].apply(
    lambda x: 1 if x == 1 else 0)

#### 2.3.1.2 Winsorize MW/employee and/or MWh/employee
To determine at which percentile to winsorize these variables to, the largest county-level carbon intensity was determined and compared for several different winsorization configurations. Also, average MWh/employee values after winsorization were compared to values provided in the literature. Wei et al. (2009) report job-years/GWh for the O&M & fuel processing for Coal and Natural Gas plants to be 0.09 and 0.08, which equate to 11,111 and 12,500 MWh/employee, respectively (https://www.sciencedirect.com/science/article/pii/S0301421509007915). Additionally, Kis et al. (2018) find the jobs/PJ/year for coal plants to be between 10-20 jobs/PJ/year (13,889-27,778 MWh/employee), and between 5-15 jobs/PJ/year (18,519-55,556 MWh/employee) for gas plants (https://www.sciencedirect.com/science/article/pii/S0301421518303239). Given these literature values, different winsorization configurations were compared based on three objectives:
1. Yield a mean MWh/employee value in the same order of magnitude as those in the literature (between 13,889 and 55,556 MWh/employee)
2. Minimize the number of datapoints altered through winsorization
3. Ensure that the maximum carbon intensity per employee after winsorization is reasonable (i.e. below 100k tonCO2e/employee).

To compute these configurations, different combinations of winsorization cutoffs for MW/employee and MWh/employee were used and new employment and carbon intensity figures were calculated, and summary statistics printed. The code below performs these operations. Note that in order to compute the carbon intensity per employee, this figure was winsorized at the 99th percentile for each iteration.

In [ ]:
# Compute MWh/employee and MW/employee for each entry
emissions2019_county_fuels_emp['MWh_per_emp'] = emissions2019_county_fuels_emp[
    'Generation (kWh)'] / 1000 / emissions2019_county_fuels_emp['emp_new']
emissions2019_county_fuels_emp['MW_per_emp'] = emissions2019_county_fuels_emp[
    'Nameplate Capacity (MW)'] / emissions2019_county_fuels_emp['emp_new']

# Separate into different dataframes based on fuel type
emissions2019_county_fuels_emp_coal = emissions2019_county_fuels_emp[
    emissions2019_county_fuels_emp['Aggregated Fuel Group'] == 'COAL'].reset_index(drop=True)
emissions2019_county_fuels_emp_ng = emissions2019_county_fuels_emp[
    emissions2019_county_fuels_emp['Aggregated Fuel Group'] == 'GAS'].reset_index(drop=True)
emissions2019_county_fuels_emp_other = emissions2019_county_fuels_emp[
    (emissions2019_county_fuels_emp['Aggregated Fuel Group'] != 'COAL') &
    (emissions2019_county_fuels_emp['Aggregated Fuel Group'] != 'GAS')].reset_index(drop=True)

In [ ]:
# Iterate over different winsorization options
from scipy.stats.mstats import winsorize

winsor_count = []
intensity_maxes = []
intensity_means = []
MWh_per_emp_means = []
cutoffs = []
run_count = 0

for cutoff_MW in [1, 0.95, 0.9, 0.8, 0.75, 0.7, 0.6]:
    for cutoff_MWh in [1, 0.95, 0.9, 0.8, 0.75, 0.7, 0.6]:
        df1, df2, df3 = emissions2019_county_fuels_emp_coal.copy(
        ), emissions2019_county_fuels_emp_ng.copy(), emissions2019_county_fuels_emp_other.copy()

        df1['MWh_per_emp'] = winsorize(np.array(df1['MWh_per_emp']), [0, 1-cutoff_MWh])
        df2['MWh_per_emp'] = winsorize(np.array(df2['MWh_per_emp']), [0, 1-cutoff_MWh])
        df3['MWh_per_emp'] = winsorize(np.array(df3['MWh_per_emp']), [0, 1-cutoff_MWh])

        df1['MW_per_emp'] = winsorize(np.array(df1['MW_per_emp']), [0, 1-cutoff_MW])
        df2['MW_per_emp'] = winsorize(np.array(df2['MW_per_emp']), [0, 1-cutoff_MW])
        df3['MW_per_emp'] = winsorize(np.array(df3['MW_per_emp']), [0, 1-cutoff_MW])

        # Add column indicating whether a row has been winsorized
        #  Create array of the maximum values in each of the winsorized columns for each df
        maxes = np.array([[max(df1['MWh_per_emp']), max(df1['MW_per_emp'])],
                          [max(df2['MWh_per_emp']), max(df2['MW_per_emp'])],
                          [max(df3['MWh_per_emp']), max(df3['MWh_per_emp'])]
                          ])

        #  Define a function to check if a given row has been winsorized by checking its MWh_per_emp and MW_per_emp values against the maximum values in the corresponding column
        def check_if_winsorized(x, i, maxes):
            if x == maxes[i]:
                return 1
            else:
                return 0

        #  Apply this function to all three dataframes
        df1['Winsorized_MWh?'] = df1.apply(
            lambda x: check_if_winsorized(x['MWh_per_emp'], 0, maxes[0]), axis=1)
        df2['Winsorized_MWh?'] = df2.apply(
            lambda x: check_if_winsorized(x['MWh_per_emp'], 0, maxes[1]), axis=1)
        df3['Winsorized_MWh?'] = df3.apply(
            lambda x: check_if_winsorized(x['MWh_per_emp'], 0, maxes[2]), axis=1)

        df1['Winsorized_MW?'] = df1.apply(
            lambda x: check_if_winsorized(x['MW_per_emp'], 1, maxes[0]), axis=1)
        df2['Winsorized_MW?'] = df2.apply(
            lambda x: check_if_winsorized(x['MW_per_emp'], 1, maxes[1]), axis=1)
        df3['Winsorized_MW?'] = df3.apply(
            lambda x: check_if_winsorized(x['MW_per_emp'], 1, maxes[2]), axis=1)

        # Re-join the dataframes of different fuels
        df = pd.concat([df1, df2, df3])

        winsor_count.append([len(df[(df['Winsorized_MWh?'] == 1) | (df['Winsorized_MW?'] == 1) | (df['Winsorized_emp?'] == 1)]),
                             len(df[(df['Winsorized_MWh?'] == 1) | (df['Winsorized_MW?'] == 1)])
                             ])
        cutoffs.append([cutoff_MW, cutoff_MWh])

        # Derive new employment figures using winsorized MWh/employee and MW/employee distributions (only apply to winsorized columns)
        df['emp_new_MWh'] = df.apply(lambda x: x['Generation (kWh)'] / 1000 /
                                     x['MWh_per_emp'] if x['Winsorized_MWh?'] == 1 else x['emp_new'], axis=1)

        df['emp_new_MW'] = df.apply(lambda x: x['Nameplate Capacity (MW)'] /
                                    x['MW_per_emp'] if x['Winsorized_MW?'] == 1 else x['emp_new'], axis=1)

        # Take largest value of emp_new_MWh and emp_new_MW, and drop these two fields afterwards
        df['emp_new'] = df.apply(lambda x: max(x.emp_new_MWh, x.emp_new_MW), axis=1)
        df = df.drop(columns=['emp_new_MWh', 'emp_new_MW'])

        # Recompute MWh/employee MW/employee based on emp_new
        df['MWh_per_emp'] = df['Generation (kWh)'] / 1000 / df['emp_new']
        df['MW_per_emp'] = df['Nameplate Capacity (MW)'] / df['emp_new']

        # print('cutoff_MW:', cutoff_MW, 'cutoff_MWh:', cutoff_MWh)
        # print('No. winsorized datapoints:', winsor_count[run_count])
        # percentiles = [0.25, 0.5, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]))
        # print(df[['MWh_per_emp', 'MW_per_emp', 'emp', 'emp_new']].describe())
        MWh_per_emp_means.append(df['MWh_per_emp'].mean())

        # Group county-level data from each fuel type into one overall employment and emissiosn figure
        df_grouped = df[['FIPS', 'County', 'State', 'Generation (kWh)', 'Useful Thermal Output (MMBtu)',
                         'Tons of CO2 Emissions', 'Metric Tonnes of CO2 Emissions', 'Nameplate Capacity (MW)',
                         'Establishments', 'emp', 'emp_new']
                        ].groupby(by=['FIPS', 'County', 'State'], as_index=False).sum()

        df_grouped['short_tonCO2e_peremployee'] = df_grouped['Tons of CO2 Emissions'] / \
            df_grouped['emp_new']
        df_grouped['short_tonCO2e_peremployee'] = winsorize(
            np.array(df_grouped['short_tonCO2e_peremployee']), [0, 0.01])

        # print('grouped:')
        # percentiles = [0.25, 0.5, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]))
        # print(df_grouped[['emp', 'emp_new', 'short_tonCO2e_peremployee']].describe())

        intensity_maxes.append(df_grouped['short_tonCO2e_peremployee'].max())
        intensity_means.append(df_grouped['short_tonCO2e_peremployee'].mean())

        run_count += 1

In [ ]:
# Print summary statistics of interest
for i in range(len(winsor_count)):
    if i > 0:
        if cutoffs[i][0] < cutoffs[i-1][0]:
            print('-'*100)
    print('Cutoffs (MW, MWh):', cutoffs[i], '\tNo. winsoried datapoints (total, excluding <1emp pts):', winsor_count[i], '\t\tMax carbon intensity:',
          intensity_maxes[i], '\t\tMean carbon intensity:', intensity_means[i], '\t\tMean MWh/employee:', MWh_per_emp_means[i])

<b>Findings & discussion from winsorization:</b> Firstly, it is clear that winsorizing MWh/employee has a greater effect on carbon intensity than MW/employee. Winsorizing MWh/employee at the 95th percentile reduces the maximum carbon intensity by ~65%, while doing the same to the MW/employee distribution yields a 53% reduction, and this trend continues as more aggressive winsorizations are trialed. Secondly, it is also apparent that there is little benefit in winsorizing both MW/employee and MWh/employee as opposed to only one of the distributions. When comparing different winsorizations of MWh/employee with MW/employee winsorized at 70% to the same winsorizations of MWh/employee but with no winsorization of MW/employee, we see that there is little change in the maximum and mean carbon intensity past the 95% winsorization of MWh/employee, but there is a material difference in the number of winsorized datapoints. From these two findings, it seems preferable to winsorize MWh/employee and to leave the MW/employee distribution intact.

From the different winsorizations of MWh/employee conducted, <b>a winsorization at the 75th percentile seems most appropriate.</b> While none of the attempted winsorizations achieve goal 3, winsorizing at 75% achieves goal 1 while reasonably minimizing the number of altered datapoints. It is clear that, in order to achieve goal 3, further winsorization is needed.

When looking at the plants within counties where MWh/emp was winsorized, the majority of these are Sectors 1 or 2 (i.e. indepedent power producers with the primary business purpose of producing power) with large capacities and unrealisitically low USEER employment figures. In the majority of cases, these plants are the only plants in their county. While there is a large spread of nameplate capacities for these facilities, up to 4 GW, the modal capacity for the winsorized plants is less than 100 MW.

By contrast, when we consider the plants in the remaining counties that still have very high carbon intensities per employee following the MWh/employee winsorization, we find that these are overwhelmingly Sector 7 (industrial plants that generate power) and are mostly low capacity (<70MW) with very low employment figures. These Sector 7 plants only represent a small fraction of total Sector 7 plants in the U.S., and are often the only plant in the county, therefore although they might not have a high absolute carbon footprint, they make the whole county have a high carbon intensity per employee. Furthermore, many of these (and other) Sector 7 plants were not winsorized in the MWh/employee winsorization, likely because they both export and import electricity to and from the grid resulting in a low net generation figure that is not necessarily proportional to their emissions.

Finally, when looking at the counties where employment was winsorized to a minimum of 1 employee, we find that these counties are overwhelmingly low capacity (<25 MW) and have only one plant in the county. Many of these counties were winsorized again by the MWh/employee winsorization.

This tells us several things. Firstly, as already discussed, it indicates that the USEER dataset underestimates employment for counties i) with only one plant in the county and/or ii) with low-capacity plants. This makes sense, given the dataset's reliance on imputation; trends identified during imputation may not hold as well at low capacities and with few plants per county, resulting in employment figures that are skewed for these counties. This validates the use of aggressive winsorizations on an MWh/employee basis. Secondly, we also find that industrial plants are particularly prone to this error, as the nature of their operation means that correlations between net generation and employment may be weak or non-existent. This points to a need for winsorization of another variable.

To deal with these errors in low-capacity, sparse counties, we will:
1. Winsorize MWh/employee at the 75th percentile for all plants;
2. Winsorize carbon intensity per employee aggressively (75th percentile) for counties where there is only one plant in the county and that plant belongs to one of Sector Codes 4, 5, 6 or 7 (i.e. a commercial or industrial plant); and
3. Winsorize the remaining counties at the 99th percentile to deal with any remaining outlier counties.

In [ ]:
# Winsorize MWh per employee and MW/employee
cutoff_MWh = 0.75
cutoff_MW = 1

emissions2019_county_fuels_emp_coal['MWh_per_emp'] = winsorize(
    np.array(emissions2019_county_fuels_emp_coal['MWh_per_emp']), [0, 1-cutoff_MWh])
emissions2019_county_fuels_emp_ng['MWh_per_emp'] = winsorize(
    np.array(emissions2019_county_fuels_emp_ng['MWh_per_emp']), [0, 1-cutoff_MWh])
emissions2019_county_fuels_emp_other['MWh_per_emp'] = winsorize(
    np.array(emissions2019_county_fuels_emp_other['MWh_per_emp']), [0, 1-cutoff_MWh])

emissions2019_county_fuels_emp_coal['MW_per_emp'] = winsorize(
    np.array(emissions2019_county_fuels_emp_coal['MW_per_emp']), [0, 1-cutoff_MW])
emissions2019_county_fuels_emp_ng['MW_per_emp'] = winsorize(
    np.array(emissions2019_county_fuels_emp_ng['MW_per_emp']), [0, 1-cutoff_MW])
emissions2019_county_fuels_emp_other['MW_per_emp'] = winsorize(
    np.array(emissions2019_county_fuels_emp_other['MW_per_emp']), [0, 1-cutoff_MW])

# Add column indicating whether a row has been winsorized
#  Create array of the maximum values in each of the winsorized columns for each df
maxes = np.array([[max(emissions2019_county_fuels_emp_coal['MWh_per_emp']),  max(emissions2019_county_fuels_emp_coal['MW_per_emp'])],
                  [max(emissions2019_county_fuels_emp_ng['MWh_per_emp']),
                   max(emissions2019_county_fuels_emp_ng['MW_per_emp'])],
                  [max(emissions2019_county_fuels_emp_other['MWh_per_emp']),
                   max(emissions2019_county_fuels_emp_other['MWh_per_emp'])]
                  ])

# Define a function to check if a given row has been winsorized by checking its
# MWh_per_emp and MW_per_emp values against the maximum values in the corresponding column
def check_if_winsorized(x, i, maxes):  # where i indicates whether we're considering MW/employee or MWh/employee
    if x == maxes[i]:
        return 1
    else:
        return 0


#  Apply this function to all three dataframes
emissions2019_county_fuels_emp_coal['Winsorized_MWh?'] = emissions2019_county_fuels_emp_coal.apply(
    lambda x: check_if_winsorized(x['MWh_per_emp'], 0, maxes[0]), axis=1)
emissions2019_county_fuels_emp_ng['Winsorized_MWh?'] = emissions2019_county_fuels_emp_ng.apply(
    lambda x: check_if_winsorized(x['MWh_per_emp'], 0, maxes[1]), axis=1)
emissions2019_county_fuels_emp_other['Winsorized_MWh?'] = emissions2019_county_fuels_emp_other.apply(
    lambda x: check_if_winsorized(x['MWh_per_emp'], 0, maxes[2]), axis=1)

emissions2019_county_fuels_emp_coal['Winsorized_MW?'] = emissions2019_county_fuels_emp_coal.apply(
    lambda x: check_if_winsorized(x['MW_per_emp'], 1, maxes[0]), axis=1)
emissions2019_county_fuels_emp_ng['Winsorized_MW?'] = emissions2019_county_fuels_emp_ng.apply(
    lambda x: check_if_winsorized(x['MW_per_emp'], 1, maxes[1]), axis=1)
emissions2019_county_fuels_emp_other['Winsorized_MW?'] = emissions2019_county_fuels_emp_other.apply(
    lambda x: check_if_winsorized(x['MW_per_emp'], 1, maxes[2]), axis=1)

# Re-join the dataframes of different fuels
emissions2019_county_fuels_emp_new = pd.concat(
    [emissions2019_county_fuels_emp_coal,
     emissions2019_county_fuels_emp_ng,
     emissions2019_county_fuels_emp_other]
)

# Derive new employment figures using winsorized MWh/employee and MW/employee distributions (only apply to winsorized columns)
emissions2019_county_fuels_emp_new['emp_new_MWh'] = emissions2019_county_fuels_emp_new.apply(
    lambda x: x['Generation (kWh)'] / 1000 /
    x['MWh_per_emp'] if x['Winsorized_MWh?'] == 1 else x['emp_new'],
    axis=1
)
emissions2019_county_fuels_emp_new['emp_new_MW'] = emissions2019_county_fuels_emp_new.apply(
    lambda x: x['Nameplate Capacity (MW)'] /
    x['MW_per_emp'] if x['Winsorized_MW?'] == 1 else x['emp_new'],
    axis=1
)

# Take largest value of emp_new_MWh and emp_new_MW, and drop these two fields afterwards
emissions2019_county_fuels_emp_new['emp_new'] = emissions2019_county_fuels_emp_new.apply(
    lambda x: max(x.emp_new_MWh, x.emp_new_MW),
    axis=1
)
emissions2019_county_fuels_emp_new = emissions2019_county_fuels_emp_new.drop(
    columns=['emp_new_MWh', 'emp_new_MW'])

# Recompute MWh/employee MW/employee based on emp_new
emissions2019_county_fuels_emp_new['MWh_per_emp'] = emissions2019_county_fuels_emp_new[
    'Generation (kWh)'] / 1000 / emissions2019_county_fuels_emp_new['emp_new']
emissions2019_county_fuels_emp_new['MW_per_emp'] = emissions2019_county_fuels_emp_new[
    'Nameplate Capacity (MW)'] / emissions2019_county_fuels_emp_new['emp_new']

# Group county-level data from each fuel type into one overall employment and emissiosn figure
emissions2019_county_emp = emissions2019_county_fuels_emp_new[
    ['FIPS', 'County', 'State', 'NERC Region', 'Generation (kWh)', 'Useful Thermal Output (MMBtu)',
     'Tons of CO2 Emissions', 'tonCO2e_PET', 'tonCO2e_GAS', 'tonCO2e_COAL', 'tonCO2e_other',
     'Metric Tonnes of CO2 Emissions', 'Nameplate Capacity (MW)', 'Establishments', 'emp',
     'emp_new', 'Winsorized_MWh?', 'Winsorized_emp?']
].groupby(by=['FIPS', 'County', 'State', 'NERC Region'], as_index=False).sum()

## 2.4 Compute emissions intensity per employee per county

In [ ]:
emissions2019_county_emp['short_tonCO2e_peremployee'] = emissions2019_county_emp['Tons of CO2 Emissions'] / \
    emissions2019_county_emp['emp_new']

# Determine which counties have only one establishment, and the sector code of the establishments within these counties
one_est_fips = list(
    emissions2019_county_emp[emissions2019_county_emp['Establishments'] == 1]['FIPS'])
one_est_fips_sectors = emissions2019.set_index('FIPS').loc[one_est_fips, :].reset_index()[
    ['FIPS', 'Sector Code']]

# Merge Sector Codes for one-establishment counties onto main df
emissions2019_county_emp1 = pd.merge(
    emissions2019_county_emp, one_est_fips_sectors, how='left', on='FIPS')

# Split main df into two dfs, on with counties with one commercial or industrial establishment and one with the remaining counties.
emissions2019_county_emp_commindsectors = emissions2019_county_emp1[
    (emissions2019_county_emp1['Sector Code'] == '7') |
    (emissions2019_county_emp1['Sector Code'] == '6') |
    (emissions2019_county_emp1['Sector Code'] == '5') |
    (emissions2019_county_emp1['Sector Code'] == '4')
].copy()
emissions2019_county_emp_other = emissions2019_county_emp1[
    (emissions2019_county_emp1['Sector Code'] != '7') &
    (emissions2019_county_emp1['Sector Code'] != '6') &
    (emissions2019_county_emp1['Sector Code'] != '5') &
    (emissions2019_county_emp1['Sector Code'] != '4')
].copy()

# For the counties with one commercial or industrial establishmnet, more aggressively winsorize carbon intensity per employee
emissions2019_county_emp_commindsectors['short_tonCO2e_peremployee'] = winsorize(
    np.array(emissions2019_county_emp_commindsectors['short_tonCO2e_peremployee']),
    [0, 0.25]
)
emissions2019_county_emp_commindsectors['Winsorized_intensity?'] = emissions2019_county_emp_commindsectors['short_tonCO2e_peremployee'].apply(
    lambda x: 1 if x == emissions2019_county_emp_commindsectors['short_tonCO2e_peremployee'].max() else 0)

# For the remaining counties, winsorize conservatively at the 99th percentile
emissions2019_county_emp_other['short_tonCO2e_peremployee'] = winsorize(
    np.array(emissions2019_county_emp_other['short_tonCO2e_peremployee']),
    [0, 0.01]
)
emissions2019_county_emp_other['Winsorized_intensity?'] = emissions2019_county_emp_other['short_tonCO2e_peremployee'].apply(
    lambda x: 1 if x == emissions2019_county_emp_other['short_tonCO2e_peremployee'].max() else 0)

# Concatenate the two dataframes to re-obtain the main df
emissions2019_county_emp = pd.concat(
    [emissions2019_county_emp_commindsectors, emissions2019_county_emp_other])

# Recalculate emp_new using new winsorized intensities
for i in range(len(emissions2019_county_emp)):
    if emissions2019_county_emp.loc[i, 'Winsorized_intensity?'] == 1:
        emissions2019_county_emp.loc[i, 'emp_new'] = emissions2019_county_emp.loc[i, 'Tons of CO2 Emissions'] / \
            emissions2019_county_emp.loc[i, 'short_tonCO2e_peremployee']

# Compute log of carbon intensity for visualization purposes
emissions2019_county_emp['short_tonCO2e_peremployee_LOG'] = np.log10(
    1+emissions2019_county_emp['short_tonCO2e_peremployee'])

# 3. Emissions per capita and emissions per employed person in each county
Use decennial census population data and total employment data from QCEW to compute both emissions per capita in each county and emissions per total employed population per county. Note that the decennial census data has race and ethnicity population data as well, which will be useful when looking at the demographic angle of this.

## 3.1 Population and emissions per capita

<b>Note:</b> Emissions data is for 2019, however we only have population data from the Decennial Census for 2020

In [ ]:
# Read in population data from decennial census using API pull script specified in a separate notebook
%run ../../empData/Scripts/CensusPopulation.ipynb
census2020_pop_race = get_census_pop_data(2020)

# For now, just consider P2_001N - Total population
census2020_pop = census2020_pop_race[['FIPS', 'P2_001N']].rename(columns={'P2_001N': 'population'})

# Merge population data onto county-level emissions/employment dataframe
emissions2019_county_emp_pop = pd.merge(emissions2019_county_emp,
                                        census2020_pop,
                                        how='left',
                                        on='FIPS')

# Compute emissions per capita for each county, and compute log1p as well
emissions2019_county_emp_pop['tonCO2e_percapita'] = emissions2019_county_emp_pop['Tons of CO2 Emissions'] / \
    emissions2019_county_emp_pop['population']
emissions2019_county_emp_pop['tonCO2e_percapita_log'] = np.log10(
    1 + emissions2019_county_emp_pop['tonCO2e_percapita'])

## 3.2 Working population and emissions per working population

In [ ]:
# Read in downloaded QCEW data file for annual average employment across all industries in 2019.
#  Datatable: https://data.bls.gov/cew/apps/table_maker/v4/table_maker.htm#type=1&year=2019&qtr=A&own=0&ind=10&supp=1
#  Documentation: https://www.bls.gov/cew/about-data/documentation-guide.htm
#  Field names: https://www.bls.gov/cew/about-data/downloadable-file-layouts/annual/naics-based-annual-layout.htm
qcew2019_totalannualemp = pd.read_csv(
    '../Input/qcew2019_totalannualemp.csv',
    dtype={'area_fips': str}
)

# own_code of 0 = all ownership types. agglvl_code = 70 = county-level, all industries
qcew2019_totalannualemp = qcew2019_totalannualemp[
    (qcew2019_totalannualemp['own_code'] == 0) &
    (qcew2019_totalannualemp['agglvl_code'] == 70)]

# Discard columns on location quotient and over-the-year change
qcew2019_totalannualemp = qcew2019_totalannualemp[
    ['area_fips', 'annual_avg_estabs', 'annual_avg_emplvl', 'total_annual_wages',
     'taxable_annual_wages', 'annual_contributions', 'annual_avg_wkly_wage', 'avg_annual_pay']
]

qcew2019_totalannualemp = qcew2019_totalannualemp.rename(
    columns={'area_fips': 'FIPS'}).reset_index(drop=True)

# Merge onto emissions2019_county_emp_pop
emissions2019_county_emp_pop = pd.merge(
    emissions2019_county_emp_pop,
    qcew2019_totalannualemp[['FIPS', 'annual_avg_emplvl']],
    how='left',
    on='FIPS'
)

# Compute emissions per employed person, and log1p
emissions2019_county_emp_pop['tonCO2e_perjob'] = emissions2019_county_emp_pop['Tons of CO2 Emissions'] / \
    emissions2019_county_emp_pop['annual_avg_emplvl']
emissions2019_county_emp_pop['tonCO2e_perjob_log'] = np.log10(
    1 + emissions2019_county_emp_pop['tonCO2e_perjob'])

# 4 Write final dataframe to csv for overall analysis

In [ ]:
emissions2019_county_emp_pop.to_csv('../Output/pwr_totalCO2_final.csv')